## 地理信息

[官方文档](https://www.osgeo.cn/geopy/)

 Nominatim 是免费的，但提供低请求限制。如果您需要进行更多的查询，可以考虑使用其他（可能是付费的）服务，例如 OpenMapQuest 或 PickPoint （这两个是提名的商业提供者，因此它们应该具有相同的数据和API）

 geopy不负责地理编码服务的数据库。如果您对服务无法完成的某些查询有问题，应该将其定向到该服务的支持团队

In [9]:
from geopy.geocoders import Nominatim
import folium
from folium import Choropleth, Circle, Marker, GeoJson
from folium.plugins import HeatMap, MarkerCluster


## 地址解析

In [16]:
geolocator = Nominatim(user_agent="mybis")

addr = "天府广场"
location = geolocator.geocode(addr)
tf_addr = (location.latitude, location.longitude)

if location != None:
	print(location.address)
	print(tf_addr)
	print(location.raw)
else:
	print('未查询到:'+addr)


天府广场, 蜀都大道人民东路, 西华门社区, 西御河街道, 青羊区, 成都市, 四川省, 610015, 中国
(30.6599157, 104.0638546)
{'place_id': 76878088, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 7417454927, 'boundingbox': ['30.6549157', '30.6649157', '104.0588546', '104.0688546'], 'lat': '30.6599157', 'lon': '104.0638546', 'display_name': '天府广场, 蜀都大道人民东路, 西华门社区, 西御河街道, 青羊区, 成都市, 四川省, 610015, 中国', 'class': 'railway', 'type': 'station', 'importance': 0.4380698768979776, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/transport_train_station2.p.20.png'}


In [18]:
# 天府广场,
# 百度地址：四川省成都市锦江区人民南路一段86号
ad1 = (30.663503, 104.072425)
# 通用地址：通过算法转换后的地址
ad2 = (30.6598575631549, 104.0634342156895)
# geopy查询地址


m_1 = folium.Map(location=ad2,
                 tiles='openstreetmap', zoom_start=15)
Marker(ad1, popup="天府广场：百度地址").add_to(m_1)
Marker(ad2, popup="天府广场：通用地址").add_to(m_1)
Marker(tf_addr, popup="天府广场：查询地址").add_to(m_1)
# Display the map
m_1

# 从上面的图可以看出，通用地址和查询地址都非常精确


## 逆地址解析

In [22]:
location = geolocator.reverse(tf_addr)
if location != None:
	print(location.address)
	print(tf_addr)
	print(location.raw)
else:
	print('未查询到:'+addr)

天府广场, 蜀都大道人民东路, 西华门社区, 西御河街道, 青羊区, 成都市, 四川省, 610015, 中国
(30.6599157, 104.0638546)
{'place_id': 76878088, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 7417454927, 'lat': '30.6599157', 'lon': '104.0638546', 'display_name': '天府广场, 蜀都大道人民东路, 西华门社区, 西御河街道, 青羊区, 成都市, 四川省, 610015, 中国', 'address': {'railway': '天府广场', 'road': '蜀都大道人民东路', 'village': '西华门社区', 'suburb': '西御河街道', 'city': '青羊区', 'state': '四川省', 'ISO3166-2-lvl4': 'CN-SC', 'postcode': '610015', 'country': '中国', 'country_code': 'cn'}, 'boundingbox': ['30.6549157', '30.6649157', '104.0588546', '104.0688546']}


## 距离测量

In [33]:
from geopy.distance import geodesic

tfgc = geolocator.geocode('天府广场')
hsgj = geolocator.geocode('成都市楚峰国际中心')
print(hsgj.raw)
print(geodesic((tfgc.latitude, tfgc.longitude), (hsgj.latitude, hsgj.longitude)).meters)


{'place_id': 162278089, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 220210595, 'boundingbox': ['35.0334679', '35.0337273', '135.7623544', '135.7630601'], 'lat': '35.03361645', 'lon': '135.7627072266606', 'display_name': '承天閣美術館, 寺町通, 相生町, 十念寺前町, 上京区, 京都市, 京都府, 602-0801, 日本', 'class': 'building', 'type': 'yes', 'importance': 0.21549815043721327}
2994018.99067703


## 和Pandas结合使用

In [63]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import pandas as pd
df = pd.DataFrame({'name': [
		# '成都市天府广场', 
		# '成都市天府三街', 
		'成都市环球中心',
		'成都市香年广场'
		]})

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df['location'] = df['name'].apply(geocode)

def getloc(loc): 
	location = geocode(loc['name'])
	return pd.Series({'latitude': location.latitude, 'longitude': location.longitude})


df_loc = df.apply(getloc, axis='columns')
# df['point'] = df['location'].apply(
#     lambda loc: tuple(loc.point) if loc else None)

df = df.join(df_loc)
df


,name,location,latitude,longitude
0,成都市环球中心,"(环球中心, 天府大道北段, 桂溪街道, 高新南区, 武侯区, 成都市, 四川省, 6100...",30.571464,104.063083
1,成都市香年广场,"(香年广场, 高新南区, 武侯区, 成都市, 四川省, 中国, (30.54724685, ...",30.547247,104.064010
